In [1]:
import pandas as pd
import re
from geopy.geocoders import Nominatim
from time import sleep
from tqdm.auto import tqdm # Biblioteca para barras de progreso

c:\Users\AzShet\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# --- 1. LEER EL ARCHIVO CSV CON LA CODIFICACIÓN CORRECTA ---
try:
    df = pd.read_csv('datos_casinos_salas.csv', encoding='utf-8')
    print("✅ Archivo 'datos_casinos_salas.csv' leído correctamente con UTF-8.")
except UnicodeDecodeError:
    print("⚠️ UTF-8 falló. Intentando con 'latin-1'...")
    df = pd.read_csv('datos_casinos_salas.csv', encoding='latin-1')
    print("✅ Archivo leído con 'latin-1'.")
except FileNotFoundError:
    print("❌ Error: No se encontró el archivo 'datos_casinos_salas.csv'.")
    data = {'Dirección': ['AV. LARCO N° 520, MIRAFLORES', 'JR. DE LA UNION No. 456 LIMA', 'AV. PRIMAVERA 123 - SURCO'],
            'Distrito': ['MIRAFLORES', 'LIMA', 'SANTIAGO DE SURCO'],
            'Provincia': ['LIMA', 'LIMA', 'LIMA'],
            'Departamento': ['LIMA', 'LIMA', 'LIMA']}
    df = pd.DataFrame(data)

✅ Archivo 'datos_casinos_salas.csv' leído correctamente con UTF-8.


In [3]:
# --- 2. GENERADOR DE VARIANTES DE DIRECCIÓN ---
def generar_variantes(row):
    """Crea una lista de posibles direcciones a partir de una fila de datos."""
    # Extraer y limpiar componentes base
    direccion = str(row['Dirección']).strip()
    distrito = str(row['Distrito']).strip()
    provincia = str(row['Provincia']).strip()
    departamento = str(row['Departamento']).strip()
    
    # Limpieza base de la dirección principal
    # Reemplaza guiones o slashes cerca de números por espacios
    base_addr = re.sub(r'\s*[-\/]\s*(?=\d)', ' ', direccion) 
    base_addr = re.sub(r'\s+', ' ', base_addr).strip()

    # Crear variantes
    variantes = []
    
    # Var 1: La más limpia (sin 'No.', 'N°', etc.)
    addr_sin_num = re.sub(r'Nro\.?|No\.?|N°|Nº', ' ', base_addr, flags=re.IGNORECASE)
    addr_sin_num = re.sub(r'\s+', ' ', addr_sin_num).strip()
    variantes.append(f"{addr_sin_num}, {distrito}, {provincia}, {departamento}, Perú")
    
    # Var 2: Añadiendo la palabra "Casino"
    variantes.append(f"Casino {addr_sin_num}, {distrito}, {provincia}, {departamento}, Perú")
    
    # Var 3: Manteniendo la nomenclatura original ('No.', 'N°', etc.)
    variantes.append(f"{base_addr}, {distrito}, {provincia}, {departamento}, Perú")

    # Var 4: Dirección simple sin departamento (a veces ayuda)
    variantes.append(f"{addr_sin_num}, {distrito}, {provincia}, Perú")

    # Eliminar duplicados y devolver la lista
    return list(pd.Series(variantes).unique())

In [ ]:
# --- 3. LÓGICA DE GEOCODIFICACIÓN INTELIGENTE ---
geolocator = Nominatim(user_agent="geocodificador_robusto_v3")
resultados_finales = []

print("\n🚀 Iniciando geocodificación con estrategia de variantes...")

# Usamos tqdm para una barra de progreso visual
for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Geocodificando"):
    
    variantes_a_probar = generar_variantes(row)
    
    # Variables para guardar el primer resultado exitoso
    latitud_encontrada, longitud_encontrada = None, None
    estado_final = 'No Encontrado'
    direccion_exitosa = None

    # Iterar sobre cada variante hasta encontrar una que funcione
    for intento_addr in variantes_a_probar:
        try:
            sleep(1) # Pausa obligatoria para no saturar el servicio
            location = geolocator.geocode(intento_addr, timeout=10)
            
            # Si se encuentra una ubicación, guardamos el resultado y rompemos el bucle
            if location:
                latitud_encontrada = location.latitude
                longitud_encontrada = location.longitude
                estado_final = 'Encontrado'
                direccion_exitosa = intento_addr
                break # ¡Éxito! No necesitamos probar más variantes para esta fila.

        except Exception as e:
            print(f"⚠️ Error de conexión en la fila {index}. Reintentando con la siguiente variante. Error: {e}")
            estado_final = f'Error de conexión'

    # Guardar el resultado final de la fila (sea exitoso o no)
    resultados_finales.append({
        'Direccion_Original': row['Dirección'],
        'Direccion_Exitosa': direccion_exitosa if direccion_exitosa else 'N/A',
        'Latitud': latitud_encontrada,
        'Longitud': longitud_encontrada,
        'Estado': estado_final
    })


🚀 Iniciando geocodificación con estrategia de variantes...


Geocodificando:   0%|          | 0/692 [00:00<?, ?it/s]

Geocodificando:   0%|          | 0/692 [00:01<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
# --- 4. CREAR Y GUARDAR EL DATAFRAME FINAL ---
df_resultado = pd.DataFrame(resultados_finales)
# Contar y mostrar resumen
encontrados = df_resultado[df_resultado['Estado'] == 'Encontrado'].shape[0]
no_encontrados = df_resultado[df_resultado['Estado'] != 'Encontrado'].shape[0]

print("\n--- Resumen Final ---")
print(f"✅ Direcciones encontradas con éxito: {encontrados}")
print(f"❌ Direcciones no encontradas (tras todos los intentos): {no_encontrados}")
print("---------------------\n")

In [ ]:
# Guardar en CSV
nombre_archivo_salida = 'resultados_geocodificados_variantes.csv'
df_resultado.to_csv(nombre_archivo_salida, index=False, encoding='utf-8-sig')

print(f"🎉 ¡Proceso completo! Resultados guardados en '{nombre_archivo_salida}'")
print("\nVista previa del resultado final:")
print(df_resultado.head())